In [22]:
from pyomo.environ import *
import pandas as pd
import numpy as np

In [23]:
def read_input(filename):
    NetworkData = pd.read_excel(filename, sheet_name= 'NetworkData', index_col=0)
    SystemDemand= pd.read_excel(filename, sheet_name = 'SystemDemand', index_col= 0)
    PVGeneration = pd.read_excel(filename, sheet_name='PVGeneration', index_col=0)
    WindGeneration = pd.read_excel(filename, sheet_name='WindGeneration', index_col=0)
    LoadData = pd.read_excel(filename, sheet_name = 'Loads', index_col= 0)
    PVData = pd.read_excel(filename, sheet_name='PVParks', index_col=0)
    WPData = pd.read_excel(filename, sheet_name='WindParks', index_col=0)
    StorageData = pd.read_excel(filename, sheet_name= 'StorageSystems', index_col=0)
    UnitData = pd.read_excel(filename, sheet_name = 'Generators', index_col= 0)
    GeneratorStepSizeData = pd.read_excel(filename, sheet_name = 'GeneratorStepSize', index_col= 0)
    GeneratorStepCostData = pd.read_excel(filename, sheet_name = 'GeneratorStepCost', index_col= 0)

    return {'NetworkData':NetworkData, 'SystemDemand':SystemDemand, 'PVGeneration':PVGeneration, 'WindGeneration':WindGeneration,
            'LoadData':LoadData, 'PVData':PVData, 'WPData':WPData,'StorageData':StorageData,'UnitData':UnitData,
            'GeneratorStepSizeData':GeneratorStepSizeData, 'GeneratorStepCostData':GeneratorStepCostData}

In [24]:
data = read_input(r'Input_Files\ValidationSystem.xlsx')
for i in data.keys():
    print(i, ':\n', data[i])
    print('\n')

NetworkData :
        FROM    TO       R       X  Capacity
Line1  Bus1  Bus2  0.1170  0.0480      1000
Line2  Bus1  Bus4  0.1073  0.0440      1000
Line3  Bus2  Bus4  0.1645  0.0457        30
Line4  Bus2  Bus3  0.1495  0.0415       100
Line5  Bus3  Bus4  0.1495  0.0415      1000


SystemDemand :
     SystemDemand
t1           180
t2           185
t3           190
t4           200
t5           215
t6            20


PVGeneration :
     PVGeneration
t1             0
t2             5
t3            10
t4            15
t5             5
t6             0


WindGeneration :
     WindGeneration
t1              10
t2               5
t3               0
t4               5
t5              10
t6              10


LoadData :
       Location  Percentage
Load1     Bus2         0.6
Load2     Bus4         0.4


PVData :
     Location  Percentage
PV1     Bus1           1


WPData :
     Location  Percentage
WP1     Bus2           1


StorageData :
       Power  Energy  SOEini  Eff Location
ESS1      5     

In [25]:
def optimization_model(inputData, Vnom, Sbase, Zbase):

    NetworkData = inputData['NetworkData']
    SystemDemand= inputData['SystemDemand']
    PVGeneration = inputData['PVGeneration']
    WindGeneration = inputData['WindGeneration']
    LoadData = inputData['LoadData']
    PVData = inputData['PVData']
    WPData = inputData['WPData']
    StorageData = inputData['StorageData']
    UnitData = inputData['UnitData']
    GeneratorStepSizeData = inputData['GeneratorStepSizeData']
    GeneratorStepCostData = inputData['GeneratorStepCostData']

    #---------------------------------------------------------------------------------------------------------
    #Define the Model
    #---------------------------------------------------------------------------------------------------------

    model = ConcreteModel()

    #---------------------------------------------------------------------------------------------------------
    #Define Sets
    #---------------------------------------------------------------------------------------------------------

    lines = list(zip(NetworkData.FROM, NetworkData.TO))
    model.LINES = Set(ordered=True, initialize=lines)
    buses = NetworkData['FROM'].append(NetworkData['TO']).unique()
    model.NODES = Set(ordered=True, initialize=buses)
    model.TIME = Set(ordered=True, initialize=SystemDemand.index)

    #---------------------------------------------------------------------------------------------------------
    #Define Parameters
    #---------------------------------------------------------------------------------------------------------

    model.R = Param(model.LINES, within=NonNegativeReals, mutable=True)
    model.X = Param(model.LINES, within=NonNegativeReals, mutable=True)

    model.Pd = Param(model.NODES, within=NonNegativeReals, mutable=True)
    model.Qd = Param(model.NODES, within=NonNegativeReals, mutable=True)
    model.Vmax = Param(within=NonNegativeReals, mutable=True)
    model.Vmin = Param(within=NonNegativeReals, mutable=True)

    #---------------------------------------------------------------------------------------------------------
    #Initialize Parameters
    #---------------------------------------------------------------------------------------------------------

    for i,j in model.LINES:
        index = model.LINES.ord((i,j))
        model.R[i,j] = NetworkData['R'].iloc[index-1]/Zbase
        model.X[i,j] = NetworkData['X'].iloc[index-1]/Zbase

    load = pd.DataFrame(index=buses, columns=('Pd', 'Qd', 'Ps', 'Qs'))
    for i in load.index:
        load.loc[i, 'Pd'] = 0
        load.loc[i, 'Qd'] = 0
    load.loc['Bus2', 'Pd'] = 0.55*180/Sbase
    load.loc['Bus2', 'Qd'] = 0.55*50/Sbase
    load.loc['Bus4', 'Pd'] = 0.45*180/Sbase
    load.loc['Bus4', 'Qd'] = 0.45*50/Sbase

    for n in model.NODES:
        model.Pd[n] = load.loc[n, 'Pd']
        model.Qd[n] = load.loc[n, 'Qd']

    model.Vmax = 1.05
    model.Vmin = 0.8

    #---------------------------------------------------------------------------------------------------------
    #Define Variables
    #---------------------------------------------------------------------------------------------------------
    model.I = Var(model.LINES, initialize=0)
    model.P = Var(model.LINES, initialize=0)
    model.Q = Var(model.LINES, initialize=0)

    def active_supply_rule(model, n):
        if model.NODES.ord(n) == 1:
            temp = 0.0
        else:
            temp = 0.0
            model.Ps[n].fixed = True
        return temp
    model.Ps = Var(model.NODES, initialize=active_supply_rule)

    def reactive_supply_rule(model, n):
        if model.NODES.ord(n) == 1:
            temp = 0.0
        else:
            temp = 0.0
            model.Qs[n].fixed = True
        return temp
    model.Qs = Var(model.NODES, initialize=reactive_supply_rule)

    def ini_voltage(model, n):
        if model.NODES.ord(n) == 1:
            temp = 1
            model.V[n].fixed = True
        else:
            temp = 1
            model.Qs[n].fixed = False
        return temp
    model.V = Var(model.NODES, initialize=ini_voltage)

    def act_loss(model):
        return sum(model.R[i,j]*(model.I[i,j]**2) for i,j in model.LINES)
    model.obj = Objective(rule=act_loss)

    def active_power_flow_rule(model, k):
        # Power_in + Power_supplied = Power_out + Power_out_lost + Power_consumed
        return sum(model.P[i,j] for i,j in model.LINES if j == k) + model.Ps[k] == \
               sum(model.P[i,j] + model.R[i,j]*(model.I[i,j]**2) for i,j in model.LINES if i == k) + model.Pd[k]
    model.active_power_flow = Constraint(model.NODES, rule=active_power_flow_rule)

    def reactive_power_flow_rule(model, k):
        # Power_in + Power_supplied = Power_out + Power_out_lost + Power_consumed
        return sum(model.Q[i,j] for i,j in model.LINES if j == k) + model.Qs[k] == \
               sum(model.Q[i,j] + model.X[i,j]*(model.I[i,j]**2) for i,j in model.LINES if i == k) + model.Qd[k]
    model.reactive_power_flow = Constraint(model.NODES, rule=reactive_power_flow_rule)

    def voltage_drop_rule(model, i, j):
        return (model.V[i]**2) - (model.V[j]**2) - 2*(model.R[i,j]*model.P[i,j] + model.X[i,j]*model.Q[i,j]) - (model.R[i,j]**2 + model.X[i,j]**2)*model.I[i,j]**2 == 0
    model.voltage_drop = Constraint(model.LINES, rule=voltage_drop_rule)

    def define_current_rule(model, i, j):
        return (model.V[j]**2)*(model.I[i,j]**2) == (model.P[i,j]**2)+(model.Q[i,j]**2)
    model.define_current = Constraint(model.LINES, rule=define_current_rule)

    def voltage_limit_rule(model, n):
        return inequality(model.Vmin, model.V[n], model.Vmax)
    model.voltage_limit = Constraint(model.NODES, rule=voltage_limit_rule)

    def current_limit_rule(model, i, j):
        return 0 <= model.I[i,j]
    model.current_limit = Constraint(model.LINES, rule=current_limit_rule)

    return model

In [26]:
Sbase = 100  # MVA
Vnom = 11    # kV
Zbase = (Vnom**2)/Sbase # Ohm
model = optimization_model(data, Vnom, Sbase, Zbase)
model.pprint()

3 Set Declarations
    LINES : Dim=0, Dimen=2, Size=5, Domain=None, Ordered=Insertion, Bounds=None
        [('Bus1', 'Bus2'), ('Bus1', 'Bus4'), ('Bus2', 'Bus4'), ('Bus2', 'Bus3'), ('Bus3', 'Bus4')]
    NODES : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=Insertion, Bounds=None
        ['Bus1', 'Bus2', 'Bus3', 'Bus4']
    TIME : Dim=0, Dimen=1, Size=6, Domain=None, Ordered=Insertion, Bounds=None
        ['t1', 't2', 't3', 't4', 't5', 't6']

6 Param Declarations
    Pd : Size=4, Index=NODES, Domain=NonNegativeReals, Default=None, Mutable=True
        Key  : Value
        Bus1 :                  0
        Bus2 : 0.9900000000000001
        Bus3 :                  0
        Bus4 :               0.81
    Qd : Size=4, Index=NODES, Domain=NonNegativeReals, Default=None, Mutable=True
        Key  : Value
        Bus1 :     0
        Bus2 : 0.275
        Bus3 :     0
        Bus4 : 0.225
    R : Size=5, Index=LINES, Domain=NonNegativeReals, Default=None, Mutable=True
        Key              : V

In [27]:
solver = SolverFactory('ipopt')
solver.solve(model, tee=True)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       78
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:       23

Total number of variables............................:       23
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 27, 'Number of variables': 23, 'Sense': 'unknown'}], 'Solver': [{'Status': 'warning', 'Message': 'Ipopt 3.12.13\\x3a Converged to a locally infeasible point. Problem may be infeasible.', 'Termination condition': 'infeasible', 'Id': 200, 'Error rc': 0, 'Time': 0.25854063034057617}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [28]:
# CHECK LOSSES+DEMAND=SUPPLY
supply = sum(value(model.Ps[n]) for n in model.NODES)
demand = sum(value(model.Pd[n]) for n in model.NODES)
losses = value(model.obj)
print([supply, demand, losses])
print([supply*Sbase, demand*Sbase, losses*Sbase])

[3.08667337324287, 1.8000000000000003, 1.2866733732718862]
[308.667337324287, 180.00000000000003, 128.66733732718862]
